# Importing the Data

**Goal**: Classify attacks assuming a multiclass classification system has alerted an attack.

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 

from sklearn.metrics import classification_report, accuracy_score

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Research/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#cleaned data without non-attack values
X = pd.read_csv(root_path+'Data/Features.csv', index_col=0)
Y = pd.read_csv(root_path+'Data/Target.csv', index_col=0)

# Feature Engineering

In [0]:
# %%capture warnings1
# cluster and score
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.cluster import adjusted_rand_score

score = []
for i in range(len(X.columns)): # loop number of features
  K = MiniBatchKMeans(n_clusters=len(Y.attack_cat.unique())-1, random_state=0)
  pred = K.fit_predict(X.iloc[:, [i]].values)
  s = adjusted_rand_score(Y[Y.columns[0]].values,pred)
  score.append(s)
  


In [5]:
# Rank the features and sort

s2 = score
np.asarray(s2)

s1 = []
for i in range(len(X.columns)):
  s1.append(i)
  
np.asarray(s1)

li = list(zip(s1, s2))

def sortSecond(val): 
    return val[1] 
  
li.sort(key = sortSecond, reverse=True) 


print("Index of Feature , adjusted_rand_score\n")
for i in range(len(X.columns)):
    print(li[i])


Index of Feature , fowlkes_mallows_score

(3, 0.8640419432745722)
(9, 0.8157320356778578)
(18, 0.6252994928788094)
(22, 0.5147108464015415)
(26, 0.5072913274164151)
(12, 0.49444769249127)
(19, 0.48030034112121656)
(27, 0.47605613214430537)
(23, 0.4738361267182002)
(1, 0.47224526084922985)
(4, 0.4721383898360001)
(28, 0.47180763688026245)
(30, 0.46733018488199846)
(13, 0.4646433097419944)
(6, 0.46440849953320423)
(32, 0.4618912368596649)
(14, 0.4555514590826053)
(15, 0.45553461170762044)
(17, 0.45025910864240154)
(7, 0.44897540739582836)
(16, 0.4424476260917735)
(29, 0.43030244862251)
(2, 0.4277246305813911)
(0, 0.2824592802101722)
(8, 0.257360541031102)
(10, 0.24974145477325502)
(5, 0.18686618395142254)
(20, 0.1318495727125251)
(33, 0.13184655232101639)
(11, 0.12159476264971067)
(36, 0.08632733541401696)
(37, 0.08450175356042952)
(42, 0.0789993827293642)
(21, 0.0640340955032938)
(39, 0.058530600818611056)
(40, 0.05677557230633726)
(41, 0.04594120237275906)
(38, 0.028736139855936736)
(2

In [0]:
# Create a copy of X dataframe with columns sorted by score

titles = []

for i in range(len(X.columns)):
  p = X.columns[li[i][0]]
  titles.append(p)


X1 = pd.DataFrame(columns=titles)

for i in range(len(X.columns)):
  X1[X1.columns[i]] = X[X.columns[li[i][0]]]




In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import accuracy_score

In [9]:
# RF w/out recursive Feature Elimination
x_train, x_test, y_train, y_test = train_test_split(X1, Y, random_state=11)
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test,y_pred, digits=4))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


                precision    recall  f1-score   support

      Analysis     0.6687    0.1729    0.2747       619
      Backdoor     0.6145    0.0899    0.1569       567
           DoS     0.3181    0.2734    0.2940      4075
      Exploits     0.6351    0.8076    0.7110     11236
       Fuzzers     0.8978    0.8660    0.8816      6127
       Generic     0.9970    0.9858    0.9914     53886
Reconnaissance     0.9231    0.7483    0.8266      3385
     Shellcode     0.6841    0.6082    0.6439       388
         Worms     0.7000    0.1842    0.2917        38

      accuracy                         0.8908     80321
     macro avg     0.7154    0.5263    0.5635     80321
  weighted avg     0.8944    0.8908    0.8880     80321



In [10]:
# %%capture warnings
# Recursive Feature Elemination from # of features to 0 and keep the accuracy score of each

accuracy = []
X2 = X1.copy()

# for i in range(len(X1.columns)-1,-1,-1):
for i in range(len(X1.columns)-1): 
  x_train, x_test, y_train, y_test = train_test_split(X1, Y, test_size=0.2, random_state=11)
  X1.drop(X1.columns[len(X1.columns)-1], axis=1, inplace=True)
  clf = rf(random_state=0, n_jobs=-1)
  clf.fit(x_train, y_train)
  y_pred = clf.predict(x_test)
  accuracy.append(accuracy_score(y_test, y_pred)) 

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff

In [11]:
index = accuracy.index(max(accuracy))
# for i in accuracy:
#   print(i)
  

# print(accuracy)

print(max(accuracy), "max accuracy")

print(index, "index")


X3 = X2.iloc[:,0:len(X.columns)-index]

0.8955288917938902 max accuracy
36 index


# Performance Analysis

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X3, Y, test_size=0.2, random_state=11)

In [13]:
from sklearn.ensemble import RandomForestClassifier as rf
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [14]:
x = list(zip(X3.columns, clf.feature_importances_))

def sortSecond(val): 
    return val[1] 
  
x.sort(key = sortSecond, reverse=True) 

for i in range(len(X3.columns)):
    print(x[i])

('sbytes', 0.3306389365754425)
('smeansz', 0.22716198542350408)
('service', 0.1707769171082987)
('Sintpkt', 0.09812472437989425)
('Sjit', 0.07353270221245402)
('dmeansz', 0.07186757883809027)
('Spkts', 0.02789715546231603)


In [15]:
clf.score(x_train, y_train) 

0.9233618388801133

In [16]:
print("Accuracy")
clf.score(x_test, y_test) 

Accuracy


0.8955288917938902

In [17]:
y_pred = clf.predict(x_test)

print("Classification\n")
print(classification_report(y_test,y_pred, digits=4))

Classification

                precision    recall  f1-score   support

      Analysis     0.5703    0.1475    0.2343       495
      Backdoor     0.8269    0.0917    0.1651       469
           DoS     0.4664    0.0986    0.1628      3235
      Exploits     0.6054    0.9194    0.7301      8944
       Fuzzers     0.8938    0.8285    0.8599      4846
       Generic     0.9963    0.9876    0.9919     43219
Reconnaissance     0.9201    0.7470    0.8246      2696
     Shellcode     0.6023    0.4953    0.5436       321
         Worms     0.6522    0.4688    0.5455        32

      accuracy                         0.8955     64257
     macro avg     0.7260    0.5316    0.5620     64257
  weighted avg     0.8976    0.8955    0.8824     64257

